In [1]:
# https://www.kaggle.com/tpmeli/insights-correlation-analysis-of-192-features
import random
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from scipy.stats import skew, kurtosis, entropy, median_absolute_deviation
from sklearn.model_selection import KFold
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)

import seaborn as sns

from sklearn.preprocessing import PolynomialFeatures, StandardScaler, Normalizer, MinMaxScaler
import sklearn as sk
data_dir = '../../../data/'

In [2]:
# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Function to calculate the log of the return
# Remember that logb(x / y) = logb(x) - logb(y)
def log_return(series):
    return np.log(series).diff()

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv(data_dir + 'train.csv')
    test = pd.read_csv(data_dir + 'test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

# Function to preprocess book data (for each stock id)
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return).fillna(0)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return).fillna(0)
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    # Dict for aggregations
    create_feature_dict = {
        'bid_price1': [np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
        'ask_price1': [np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
        'bid_price2': [np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
        'ask_price2': [np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
        'wap1': [np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
        'wap2': [np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
        'log_return1': [realized_volatility, np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std, skew, kurtosis, median_absolute_deviation],
        'wap_balance': [np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
        'price_spread': [np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
        'bid_spread': [np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
        'ask_spread': [np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
        'total_volume':[np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
        'volume_imbalance':[np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_450 = get_stats_window(seconds_in_bucket = 450, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_150 = get_stats_window(seconds_in_bucket = 150, add_suffix = True)
    
    # Merge all
    df_feature = df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_150, how = 'left', left_on = 'time_id_', right_on = 'time_id__150')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150'], axis = 1, inplace = True)
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby(['time_id'])['price'].apply(log_return).fillna(0)
    
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility, np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
        'seconds_in_bucket':[count_unique],
        'size': [np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
        'order_count':[np.min, np.max, np.mean, np.median, np.std, np.sum, skew, median_absolute_deviation],
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_450 = get_stats_window(seconds_in_bucket = 450, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_150 = get_stats_window(seconds_in_bucket = 150, add_suffix = True)

    # Merge all
    df_feature = df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_150, how = 'left', left_on = 'time_id_', right_on = 'time_id__150')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150'], axis = 1, inplace = True)
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

# Function to get group stats for the stock_id and time_id
# def get_time_stock(df):
#     # Get realized volatility columns
# #     vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_450', 'log_return2_realized_volatility_450', 
# #                 'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_150', 'log_return2_realized_volatility_150', 
# #                 'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_450', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_150']
#     vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'trade_log_return_realized_volatility']

#     # Group by the stock id
# #     df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
# #     # Rename columns joining suffix
# #     df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
# #     df_stock_id = df_stock_id.add_suffix('_' + 'stock')

#     # Group by the time id
#     df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
#     # Rename columns joining suffix
#     df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
#     df_time_id = df_time_id.add_suffix('_' + 'time')
    
#     # Merge with original dataframe
#     #df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
#     df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
#     df.drop(['time_id__time'], axis = 1, inplace = True)
#     return df
    
# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df


# Read train and test
train, test = read_train_test()

# Get unique stock ids 
train_stock_ids = train['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
train_ = preprocessor(train_stock_ids, is_train = True)
train = train.merge(train_, on = ['row_id'], how = 'left').dropna()


# Get group stats of time_id and stock_id
# train = get_time_stock(train)

train.head()

Our training set has 428932 rows


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed: 29.7min finished


,stock_id,time_id,target,row_id,bid_price1_amin,bid_price1_amax,bid_price1_mean,bid_price1_median,bid_price1_std,bid_price1_sum,bid_price1_skew,bid_price1_median_absolute_deviation,ask_price1_amin,ask_price1_amax,ask_price1_mean,ask_price1_median,ask_price1_std,ask_price1_sum,ask_price1_skew,ask_price1_median_absolute_deviation,bid_price2_amin,bid_price2_amax,bid_price2_mean,bid_price2_median,bid_price2_std,bid_price2_sum,bid_price2_skew,bid_price2_median_absolute_deviation,ask_price2_amin,ask_price2_amax,ask_price2_mean,ask_price2_median,ask_price2_std,ask_price2_sum,ask_price2_skew,ask_price2_median_absolute_deviation,wap1_amin,wap1_amax,wap1_mean,wap1_median,wap1_std,wap1_sum,wap1_skew,wap1_median_absolute_deviation,wap2_amin,wap2_amax,wap2_mean,wap2_median,wap2_std,wap2_sum,wap2_skew,wap2_median_absolute_deviation,log_return1_realized_volatility,log_return1_amin,log_return1_amax,log_return1_mean,log_return1_median,log_return1_std,log_return1_sum,log_return1_skew,log_return1_median_absolute_deviation,log_return2_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_skew,log_return2_kurtosis,log_return2_median_absolute_deviation,wap_balance_amin,wap_balance_amax,wap_balance_mean,wap_balance_median,wap_balance_std,wap_balance_sum,wap_balance_skew,wap_balance_median_absolute_deviation,price_spread_amin,price_spread_amax,price_spread_mean,price_spread_median,price_spread_std,price_spread_sum,price_spread_skew,price_spread_median_absolute_deviation,bid_spread_amin,bid_spread_amax,bid_spread_mean,bid_spread_median,bid_spread_std,bid_spread_sum,bid_spread_skew,bid_spread_median_absolute_deviation,ask_spread_amin,ask_spread_amax,ask_spread_mean,ask_spread_median,ask_spread_std,ask_spread_sum,ask_spread_skew,ask_spread_median_absolute_deviation,total_volume_amin,total_volume_amax,total_volume_mean,total_volume_median,total_volume_std,total_volume_sum,total_volume_skew,total_volume_median_absolute_deviation,volume_imbalance_amin,volume_imbalance_amax,volume_imbalance_mean,volume_imbalance_median,volume_imbalance_std,volume_imbalance_sum,volume_imbalance_skew,volume_imbalance_median_absolute_deviation,bid_price1_amin_450,bid_price1_amax_450,bid_price1_mean_450,bid_price1_median_450,bid_price1_std_450,bid_price1_sum_450,bid_price1_skew_450,bid_price1_median_absolute_deviation_450,ask_price1_amin_450,ask_price1_amax_450,ask_price1_mean_450,ask_price1_median_450,ask_price1_std_450,ask_price1_sum_450,ask_price1_skew_450,ask_price1_median_absolute_deviation_450,bid_price2_amin_450,bid_price2_amax_450,bid_price2_mean_450,bid_price2_median_450,bid_price2_std_450,bid_price2_sum_450,bid_price2_skew_450,bid_price2_median_absolute_deviation_450,ask_price2_amin_450,ask_price2_amax_450,ask_price2_mean_450,ask_price2_median_450,ask_price2_std_450,ask_price2_sum_450,ask_price2_skew_450,ask_price2_median_absolute_deviation_450,wap1_amin_450,wap1_amax_450,...,wap_balance_mean_150,wap_balance_median_150,wap_balance_std_150,wap_balance_sum_150,wap_balance_skew_150,wap_balance_median_absolute_deviation_150,price_spread_amin_150,price_spread_amax_150,price_spread_mean_150,price_spread_median_150,price_spread_std_150,price_spread_sum_150,price_spread_skew_150,price_spread_median_absolute_deviation_150,bid_spread_amin_150,bid_spread_amax_150,bid_spread_mean_150,bid_spread_median_150,bid_spread_std_150,bid_spread_sum_150,bid_spread_skew_150,bid_spread_median_absolute_deviation_150,ask_spread_amin_150,ask_spread_amax_150,ask_spread_mean_150,ask_spread_median_150,ask_spread_std_150,ask_spread_sum_150,ask_spread_skew_150,ask_spread_median_absolute_deviation_150,total_volume_amin_150,total_volume_amax_150,total_volume_mean_150,total_volume_median_150,total_volume_std_150,total_volume_sum_150,total_volume_skew_150,total_volume_median_absolute_deviation_150,volume_imbalance_amin_150,volume_imbalance_amax_150,volume_imbalance_mean_150,volume_imbalance_median_150,volume_imbalance_std_150,volume_imbalance_sum_150,volume_imbalance_skew_150,v

In [3]:
print(train.isna().sum().sum())
train.columns[train.isna().sum() > 0]
train.head()

0


,stock_id,time_id,target,row_id,bid_price1_amin,bid_price1_amax,bid_price1_mean,bid_price1_median,bid_price1_std,bid_price1_sum,bid_price1_skew,bid_price1_median_absolute_deviation,ask_price1_amin,ask_price1_amax,ask_price1_mean,ask_price1_median,ask_price1_std,ask_price1_sum,ask_price1_skew,ask_price1_median_absolute_deviation,bid_price2_amin,bid_price2_amax,bid_price2_mean,bid_price2_median,bid_price2_std,bid_price2_sum,bid_price2_skew,bid_price2_median_absolute_deviation,ask_price2_amin,ask_price2_amax,ask_price2_mean,ask_price2_median,ask_price2_std,ask_price2_sum,ask_price2_skew,ask_price2_median_absolute_deviation,wap1_amin,wap1_amax,wap1_mean,wap1_median,wap1_std,wap1_sum,wap1_skew,wap1_median_absolute_deviation,wap2_amin,wap2_amax,wap2_mean,wap2_median,wap2_std,wap2_sum,wap2_skew,wap2_median_absolute_deviation,log_return1_realized_volatility,log_return1_amin,log_return1_amax,log_return1_mean,log_return1_median,log_return1_std,log_return1_sum,log_return1_skew,log_return1_median_absolute_deviation,log_return2_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_skew,log_return2_kurtosis,log_return2_median_absolute_deviation,wap_balance_amin,wap_balance_amax,wap_balance_mean,wap_balance_median,wap_balance_std,wap_balance_sum,wap_balance_skew,wap_balance_median_absolute_deviation,price_spread_amin,price_spread_amax,price_spread_mean,price_spread_median,price_spread_std,price_spread_sum,price_spread_skew,price_spread_median_absolute_deviation,bid_spread_amin,bid_spread_amax,bid_spread_mean,bid_spread_median,bid_spread_std,bid_spread_sum,bid_spread_skew,bid_spread_median_absolute_deviation,ask_spread_amin,ask_spread_amax,ask_spread_mean,ask_spread_median,ask_spread_std,ask_spread_sum,ask_spread_skew,ask_spread_median_absolute_deviation,total_volume_amin,total_volume_amax,total_volume_mean,total_volume_median,total_volume_std,total_volume_sum,total_volume_skew,total_volume_median_absolute_deviation,volume_imbalance_amin,volume_imbalance_amax,volume_imbalance_mean,volume_imbalance_median,volume_imbalance_std,volume_imbalance_sum,volume_imbalance_skew,volume_imbalance_median_absolute_deviation,bid_price1_amin_450,bid_price1_amax_450,bid_price1_mean_450,bid_price1_median_450,bid_price1_std_450,bid_price1_sum_450,bid_price1_skew_450,bid_price1_median_absolute_deviation_450,ask_price1_amin_450,ask_price1_amax_450,ask_price1_mean_450,ask_price1_median_450,ask_price1_std_450,ask_price1_sum_450,ask_price1_skew_450,ask_price1_median_absolute_deviation_450,bid_price2_amin_450,bid_price2_amax_450,bid_price2_mean_450,bid_price2_median_450,bid_price2_std_450,bid_price2_sum_450,bid_price2_skew_450,bid_price2_median_absolute_deviation_450,ask_price2_amin_450,ask_price2_amax_450,ask_price2_mean_450,ask_price2_median_450,ask_price2_std_450,ask_price2_sum_450,ask_price2_skew_450,ask_price2_median_absolute_deviation_450,wap1_amin_450,wap1_amax_450,...,wap_balance_mean_150,wap_balance_median_150,wap_balance_std_150,wap_balance_sum_150,wap_balance_skew_150,wap_balance_median_absolute_deviation_150,price_spread_amin_150,price_spread_amax_150,price_spread_mean_150,price_spread_median_150,price_spread_std_150,price_spread_sum_150,price_spread_skew_150,price_spread_median_absolute_deviation_150,bid_spread_amin_150,bid_spread_amax_150,bid_spread_mean_150,bid_spread_median_150,bid_spread_std_150,bid_spread_sum_150,bid_spread_skew_150,bid_spread_median_absolute_deviation_150,ask_spread_amin_150,ask_spread_amax_150,ask_spread_mean_150,ask_spread_median_150,ask_spread_std_150,ask_spread_sum_150,ask_spread_skew_150,ask_spread_median_absolute_deviation_150,total_volume_amin_150,total_volume_amax_150,total_volume_mean_150,total_volume_median_150,total_volume_std_150,total_volume_sum_150,total_volume_skew_150,total_volume_median_absolute_deviation_150,volume_imbalance_amin_150,volume_imbalance_amax_150,volume_imbalance_mean_150,volume_imbalance_median_150,volume_imbalance_std_150,volume_imbalance_sum_150,volume_imbalance_skew_150,v

In [91]:
train = train.dropna()

In [86]:
# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Function to calculate the log of the return
# Remember that logb(x / y) = logb(x) - logb(y)
def log_return(series):
    return np.log(series).diff()

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv(data_dir + 'train.csv')
    test = pd.read_csv(data_dir + 'test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

# Function to preprocess book data (for each stock id)
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    if df.isna().sum().sum() > 0:
        print(file_path)
    return df

# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    if df.isna().sum().sum() > 0:
        print(file_path)
    return df

    
# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        book_preprocessor(file_path_book)
        trade_preprocessor(file_path_trade)
        
        # Return the merge dataframe
        return "Hello"
    
    # Use parallel api to call paralle for loop
    # df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    for stock_id in list_stock_ids:
        for_joblib(stock_id)
    # Concatenate all the dataframes that return from Parallel
    # df = pd.concat(df, ignore_index = True)
    return df


# Read train and test
train, test = read_train_test()

# Get unique stock ids 
train_stock_ids = train['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
train_ = preprocessor(train_stock_ids, is_train = True)

Our training set has 428932 rows


In [55]:
df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).count()

,seconds_in_bucket,price,size,order_count,log_return
time_id,,,,,
5,14,14,14,14,14
11,10,10,10,10,10
16,9,9,9,9,9
31,3,3,3,3,3
62,4,4,4,4,4
...,...,...,...,...,...
32751,19,19,19,19,19
32753,5,5,5,5,5
32758,8,8,8,8,8


In [11]:
len(df.columns)

449

In [12]:
df.isna().sum().sum()

0